# Fidelity estimation: Minimax Method
We show how to use the code for minimax method using some examples.

## Example 1: Cluster state and Pauli measurements
To demonstrate how to use the minimax method to estimate fidelity, we consider a 3-qubit linear cluster state as our target state.\
We will focus on Pauli measurements.

The minimax method constructs an estimator when the target state, the measurement settings, and the confidence level is specified. One we construct the estimator, we can repeatedly reuse the estimator for the chosen settings.

We outline the basic steps that we will go through in this example - from specifying the settings to computing the estimate.
1. Create a YAML file describing the settings required to construct the estimator.
2. Construct the estimator using the specified settings.
3. Create a CSV file containing the measurement outcomes.
4. Compute the fidelity estimate using the CSV file and the constructed estimator.

We remark that step 2 is usually computation intensive. Nevertheless, once the estimator has been constructed, the fidelity estimates (step 4) can be obtained almost instantaneously.

Let's look at each step in more detail.

## Step 1: Create a YAML file with the measurement settings.
[YAML](https://en.wikipedia.org/wiki/YAML) is a markup language that is human-readable, and can be parsed by a computer. The combination of these attributes makes it a good medium to specify the settings to the code.

We allow for different ways to specify the settings in the YAML file.\
For example, one could specify the target state as a list in the YAML file, or provide a path to a `.npy` file containing the numpy array for the target state.\
Since Pauli measurements and some special states like the stabilizer states are commonly used, we have provided a special interface to conveniently specify these settings.\
We will be using the latter interface in this demo for convenience. For details on all available formats to specify the settings, we encourage the reader to refer to the documentation of the code.

We create the following settings file for the cluster state. We specify those Pauli operators $P$ that have a non-zero weight $\text{Tr}(P \rho)$.

-------------------------------------------------
### cluster_state_settings.yaml
```
target:
    - cluster: 3
POVM_list:
    - pauli: [IZX, XIX, XZI, YXY, YYZ, ZXZ, ZYY]
R_list: 100
confidence_level: 0.95
algorithm: cvxpy
```
-------------------------------------------------

Let's take a closer look at the settings.
- `target` refers to the target state. We can conveniently provide a linear cluster state using the syntax: `- cluster: nq`, where `nq` is the number of qubits.\
  We have therefore specified a 3-qubit cluster state.
  
- `POVM_list` is a list of POVMs that will measured in order to estimate the fidelity.\
  Pauli measurements can be specified in a few different ways, but here we use the most obvious one: list the Pauli operators that you want to measure.\
  The default measurement is projection on each eigenvector of the Pauli operator, but if collective measurement on eigenspace with $+1$ and $-1$ eigenvalue is required, you can include the keyword `subspace` after listing all the Pauli operators.

- `R_list` corresponds to the number of outcomes recorded for each POVM.\
    We want 100 outcomes for each Pauli measurement, so we simply write 100.\
    If something more specific is required, write a list of outcomes, one for each Pauli measurement.

- `confidence_level` should be a number between 0.75 and 1, and it determines the confidence level of the computed risk.

- `algorithm` should be either `saddle_point` (default) or `cvxpy`.\
  `saddle_point` implements accelerated projected gradient descent, which is slower than `cvxpy` but can handle higher dimensional matrices.\
    For demonstration, we choose the `cvxpy` algorithm for this example, as it runs faster.
----------
> **It is important to adhere to the syntax specified in the documentation when creating the YAML file.**\
The code is expected to throw an error when incorrect syntax is used. However, there could be some cases that slip past the sanity checks, and the code may end up constructing an estimator that was not intended by the user!

## Step 2: Construct the estimator
In order to construct the estimator, we use the function ```construct_fidelity_estimator``` included in ```handle_fidelity_estimation.py``` module.

The syntax for this function is pretty straightforward:

----------------------
```
construct_fidelity_estimator(yaml_filename, estimator_filename,
                             yaml_file_dir = './yaml_files', estimator_dir = './estimator_files')
```
----------------------

A closer look at the options:
- ```yaml_filename``` refers to the name of the YAML settings file.
- ```estimator_filename``` refers to the name of the (JSON) file to which the constructed estimator is saved.
- ```yaml_file_dir``` specifies the directory in which the YAML settings file is stored.\
  This is an optional argument, and if nothing is specified, the code assumes that the YAML file lives in a sub-directory named `yaml_files` of the current directory.
- ```estimator_dir``` specifies the directory where the constructed estimator is saved.\
  As before, this is an optional argument, and the default location is assumed to be a sub-directory named `estimator_files` of the current directory.

------------------

> We save the estimator because the same estimator can be re-used later for the same settings.\
The estimator is saved as a JSON file. These files are internally handled by the functions in the module, and need not be edited manually by the user.

------------------

Following the default options, we have created a subdirectory called `yaml_files` and placed `cluster_state_settings.yaml` YAML file there. Let us now construct the estimator. 

> **It can take anywhere from a few minutes to many hours to compute the estimator depending on the settings that were specified.**\
  If the dimension of the system is large or many measurement settings are begin used, please consider running the code on a workstation or a cluster.
  
The following code is expected to run in about 4 minutes on a laptop, though the actual time may vary depending on the hardware and the OS.

In [2]:
import project_root # adds the root directory of the project to Python Path
from handle_fidelity_estimation import construct_fidelity_estimator

construct_fidelity_estimator(yaml_filename = 'cluster_state_settings.yaml',\
                             estimator_filename = 'cluster_state_estimator.json')

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) Jun 05 09:29:51 PM: Your problem has 128 variables, 5 constraints, and 0 parameters.
(CVXPY) Jun 05 09:29:51 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jun 05 09:29:51 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jun 05 09:29:51 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jun 05 09:29:51 PM: Compiling problem (target solver=SCS).
(CVXPY) Jun 05 09:29:51 PM: Reduction chain: Complex2Real -> FlipObjective -> Dcp2Cone -> CvxAtt

/home/akshay/Work/Projects/Quantum_Information/Fidelity_Estimation/Github/minimax-fidelity-estimation/handle_fidelity_estimation.py:453: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if POVM_list[0][0] == "rpm":
/home/akshay/Work/Projects/Quantum_Information/Fidelity_Estimation/Github/minimax-fidelity-estimation/src/fidelity_estimation_cvxpy.py:148: UserWarning: The initial condition for the necessary CVXPY solvers are fixed. The `random_init` parameter will be unused.
  warnings.warn("The initial condition for the necessary CVXPY solvers are fixed. The `random_init` parameter will be unused.")


(CVXPY) Jun 05 09:29:51 PM: Applying reduction CvxAttr2Constr
(CVXPY) Jun 05 09:29:51 PM: Applying reduction ConeMatrixStuffing
(CVXPY) Jun 05 09:29:52 PM: Applying reduction SCS
(CVXPY) Jun 05 09:29:52 PM: Finished problem compilation (took 7.281e-01 seconds).
-------------------------------------------------------------------------------
                                Numerical solver                               
-------------------------------------------------------------------------------
(CVXPY) Jun 05 09:29:52 PM: Invoking solver SCS  to obtain a solution.
------------------------------------------------------------------
	       SCS v3.2.0 - Splitting Conic Solver
	(c) Brendan O'Donoghue, Stanford University, 2012
------------------------------------------------------------------
problem:  variables n: 263, constraints m: 466
cones: 	  z: primal zero / dual free vars: 4
	  l: linear vars: 1
	  q: soc vars: 168, qsize: 56
	  s: psd vars: 272, ssize: 2
	  e: exp vars: 21, dual

> 1. Note that `construct_fidelity_estimator` prints the progress of optimization by default.\
     If you wish to turn this off, supply `print_progress = False` as an additional argument to the function.
2. If an estimator file already exists, `construct_fidelity_estimator` function will throw an error. You can delete the existing estimator, move it to a different directory, or use another name to save the estimator in that case.

## Step 3: Generate measurement outcomes
We generate the measurement outcomes separately and store them in a CSV file.\
Note that these outcomes are generated using the state obtained by applying 10% depolarizing noise to the target state.

> In practice, these outcomes will come from experiments.

The CSV file looks as follows:

---------------------
### cluster_state_outcomes.csv

|     |   |   |     |   |
| --- | - | - | --- | - |
| IZX | 7 | 0 | ... | 7 |
| XIX | 7 | 7 | ... | 0 |
|  .  | . | . |  .  | . |
| ZYY | 5 | 3 | ... | 0 |

---------------------
The first column contains the labels of the Pauli measurements performed.\
Corresponding to each Pauli operator, we store the measurement outcomes in the same row as the Pauli operator.\
Outcome $i$ points to the eigenvector $\vert i \rangle$ that was observed upon measurement.

> 1. **It is important that the order of eigenvectors used for outcomes matches the POVM that was specified for constructing the estimator.**\
  We use the following convention for the eigenvectors: $\vert+++\rangle$, $\vert++-\rangle$, $\vert+-+\rangle$, ..., $\vert---\rangle$.\
  Basically, we use the binary expansion of numbers from $0$ to $2^{n_q} - 1$, where $n_q$ are the number of qubits, with $0$ replaced by $+$ and $1$ replaced by $-$.
2. **It is important that the outcomes for Pauli operators are listed in the same order as what we used for constructing the estimator.**\
   That is, we must have outcomes for IZXZ, XIX, ..., ZYY in that order in the CSV file.
  
Note for any Pauli operator $P = X_1 \dotsb X_{n_q}$, a $+$ at the $i$th qubit location means that we are looking at the $+1$ eigenvector of $X_i$, where $X_i \in \{I, X, Y, Z\}$.

> In practice, steps 2 & 3 can occur in any order.

## Step 4: Compute the fidelity estimate
Let's use the estimator that we constructed in step 2 and the outcomes generated in step 3 to compute the fidelity estimate.\
This task is handled by `compute_fidelity_estimate_risk` function in `handle_fidelity_estimation.py` module.

This function takes the following form.

----------------
```
compute_fidelity_estimate_risk(outcomes, estimator_filename, estimator_dir = './estimator_files')
```
----------------

The options accept the following formats:
- `outcomes` can be one of the following:
    1. A list of outcomes for each POVM measurement.
    2. Path to a YAML file containing a list of outcomes for each POVM measurement.
    3. - Path to a CSV file, or
       - A dictionary:\
           `{'csv_file_path': Path to CSV file, 'entries': 'row'/'column', 'start': (row index, column index)}`\
         where `row` (`column`) is used if data is stored in rows (columns),\
         and `start` denotes the index of the cell where the data starts (we start the row and column at index 0).
- `estimator_filename` is the name of the estimator file that we constructed previously.
- `estimator_dir` refers to the directory in which the estimator file has been saved.

We refer the reader to the documentation of the code which elaborates these options further.

As we can see from the CSV file outline in step 3, the data starts at the first row and the second column.\
The first column describes the data, but is not actually a part of it. Therefore, we set `start = (0, 1)`.\
As noted earlier, we label the rows and columns starting from 0, following Python convention.\
Also, it is clear that the data is stored row-wise, so we set `entries = 'row'`.

Note that we have saved the `cluster_state_outcomes.csv` file in a subdirectory called `outcome_files`.\
Using this, we compute the estimate as follows.

In [3]:
import project_root # adds the root directory of the project to Python Path
from handle_fidelity_estimation import compute_fidelity_estimate_risk

compute_fidelity_estimate_risk(outcomes = {'csv_file_path': './outcome_files/cluster_state_outcomes.csv',\
                                           'entries': 'row', 'start': (0, 1)},\
                               estimator_filename = 'cluster_state_estimator.json')

Fidelity estimate: 0.91
Risk: 0.086


We can see that the estimate $\widehat{F} \approx 0.91$ is close to the actual fidelity $F = 0.9125$.\
The risk can be reduced by increasing the number of shots and/or the Pauli measurements performed.

--------------------
--------------------

## Example 2: The Bell State and Randomized Pauli Measurement scheme

Suppose that our target state $\rho$ is the two-qubit Bell state
\begin{align}
    \rho &= \vert \psi \rangle \langle \psi \vert \\
    \text{where} \quad \vert \psi \rangle &= \frac{1}{\sqrt{2}} \left(\vert 00 \rangle + \vert 11 \rangle\right)
\end{align}

Observe that $\vert \psi \rangle$ is a stabilizer state that is generated by the stabilizers $XX$ and $ZZ$.

To compute the fidelty, we use the minimax optimal measurement scheme for stabilizer states. This amounts to sampling uniformly from the stabilizer group elements (except the identity) and recording their measurement outcome ($\pm 1).

Let's compute the estimator given by the minimax method for such a setting.

We know that
\begin{equation}
    R = \left\lceil 2\frac{\ln\left(2/\epsilon\right)}{\left|\ln\left(1 - \left(\frac{d}{d - 1}\right)^2 \widehat{R}_*^2\right)\right|} \right\rceil
\end{equation}
outcomes are sufficient to achieve a risk $\widehat{\mathcal{R}}_* \in (0, 0.5)$ with a confidence level of $1 - \epsilon \in (0.75, 1)$.

As before, we break down the process of constructing an estimator & computing an estimate into four steps:
1. Create a YAML file describing the settings to construct the estimator and the risk.
2. Construct the estimator for the specified settings.
3. Store the outcomes in a YAML file. Convert outcomes to indices in case they are eigenvalues.
4. Use the outcomes and constructed estimator to compute the fidelity estimate (and the risk).

## Step 1: Create the YAML file containing the settings
The YAML file looks as follows.

### bell_state_settings.yml
```
target:
    - stabilizer: [XX, ZZ]
POVM_list:
    - pauli: [RPM]
R_list: 1657
confidence_level: 0.95
```

We describe the a couple of above options in more detail:
- The general syntax for specifying a target stabilizer state is `- stabilizer: list of stabilizer generators`. Note that we can include a sign in front of the Pauli operator if necessary.\
  For example, we specify a stabilizer state above with $XX$ and $ZZ$ as the stabilizer generators. We could as well have used $XX$ and $-YY$ as the stabilizer generators.
- We have included a shortcut to specify the Randomized Pauli Measurement (RPM) scheme described in section II.E. of the PRA submission. The syntax is always `- pauli: [RPM]` for specifying this measurement scheme.\
  For stabilizer states, this amounts to randomly sampling the stabilizer group (excluding the identity) and recording the eigenvalues of outcomes.

Note that we use a confidence level of $95\%$ and a risk of $\widehat{\mathcal{R}}_* = 0.05$ to obtain $R = 1657$.

## Step 2: Construct the estimator using the YAML settings file
As before, we use the function ```construct_fidelity_estimator``` in ```handle_fidelity_estimation.py``` module to construct the estimator.

We have placed `bell_state_settings.yaml` settings file in the `yaml_files` subdirectory.

> The estimator for the RPM measurement scheme is constructed efficiently. It should take at most a few minutes, if not seconds, to construct the estimator.

In [1]:
import project_root # adds the root directory of the project to Python Path
from handle_fidelity_estimation import construct_fidelity_estimator

construct_fidelity_estimator(yaml_filename = 'bell_state_settings.yaml',\
                             estimator_filename = 'bell_state_estimator.json')

Optimization complete 


You can check that there is a subdirectory called `estimator_files` (if it wasn't already there), and you can find the file `bell_state_estimator.json` there.

## Step 3: Create a YAML file with the measurement outcomes

We created some outcomes beforehand to test the estimator. For this purpose, we added $10\%$ depolarizing noise to the target state $\rho$, and then performed the Pauli measurements as prescibed by Randomized Pauli Measurement (RPM) scheme.

Note that for the RPM scheme, only the *number* of $+1$ and $-1$ eigenvalues are important. It doesn't matter which Pauli measurement gave a $+1$ outcome or a $-1$ outcome.\
Before we supply the outcomes to the estimator, we need to convert $+1 \to 0$ and $-1 \to 1$. The reason is that the estimator works by referring to the POVM elements and we used $\{E_+, E_-\}$ as the POVM when constructing the estimator, in that order.\
Because the outcomes are going be just $0$ and $1$, we put the outcomes in a list inside a YAML file.

> For the sake of demonstration, this time we choose to save our outcomes in a YAML file instead of a CSV file.\
  A CSV file can be used if that's preferred.

The YAML file containing the outcomes looks as follows.
### bell_state_measurement_outcomes.yaml
```
outcomes:
    - [0, 0, ...]
```
Note that there must be exactly $R = 1657$ measurement outcomes, because the estimator was constructed for this case.

> The syntax used in the YAML file is important for ensuring proper parsing of the file.\
  The code documentation can be referred for details.

We use these outcomes to compute the fidelity estimate.

## Step 4: Compute the fidelity estimate
We now supply the outcomes to the `compute_fidelity_estimate_risk` function in `handle_fidelity_estimation.py` module.

We have saved the `bell_state_measurement_outcomes.yml` file in a subdirectory called `outcome_files`, and we use this to compute the estimate.

In [4]:
import project_root # adds the root directory of the project to Python Path
from handle_fidelity_estimation import compute_fidelity_estimate_risk

compute_fidelity_estimate_risk(outcomes = './outcome_files/bell_state_measurement_outcomes.yaml',\
                               estimator_filename = 'bell_state_estimator.json')

Fidelity estimate: 0.933
Risk: 0.05


Observe that the risk is very close to the value of $0.05$ (only a rounded value is displayed) that we chose to determine the number of outcomes $R = 1657$.

We can also see that the fidelity estimate $\widehat{F} \approx 0.933$ is close to the actual fidelity $F = 0.925$, and within the specified risk of $0.05$.

This estimate can be found in Table II of PRA submission.

--------------------
--------------------

# Epilogue

Other formats are supported by the YAML settings file. You can directly supply lists to it or give a path to a `.npy` file which contains the array describing your target state or POVMs. Please read the documentation to see all the available options.

Note that the code can be run directly from the commandline. This is especially helpful if one needs to run the code on a cluster or even on a workstation. Please refer the documentation for details on how to use this functionality.